### Importing the needed libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn import preprocessing
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler #you can use minmax scaler too
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import xgboost as xgb
import imblearn
from imblearn.ensemble import BalancedBaggingClassifier
#Import other necessary model libraries, for this example, using Logistic Regression

### Importing the dataset

In [139]:
train_data = pd.read_csv(r"../Data/Preprocessed-Datasets/train-data-encoded.csv", header = 0)
test_data = pd.read_csv(r"../Data/Preprocessed-Datasets/test-data-encoded.csv", header = 0)
validation_data = pd.read_csv(r"../Data/Preprocessed-Datasets/validation-data-encoded.csv", header = 0)

In [3]:
train_data=train_data.drop(['Reservation-id'], axis = 1)
validation_data=validation_data.drop(['Reservation-id'], axis = 1)
test_data=test_data.drop(['Reservation-id'], axis = 1)

In [140]:
test_data.head()

,Reservation-id,Age,Adults,Children,Babies,Discount_Rate,Room_Rate,Expected_stay_days,Reservation_gap,Gender_F,...,Deposit_type_No Deposit,Deposit_type_Non-Refundable,Deposit_type_Refundable,Booking_channel_Agent,Booking_channel_Direct,Booking_channel_Online,Required_Car_Parking_No,Required_Car_Parking_Yes,Use_Promotion_No,Use_Promotion_Yes
0,62931593,52,3,3,0,10,153,1,21,1,...,1,0,0,0,1,0,0,1,0,1
1,70586099,47,2,1,0,0,210,1,104,1,...,1,0,0,0,0,1,1,0,1,0
2,4230648,28,2,2,0,5,117,3,20,1,...,1,0,0,1,0,0,1,0,0,1
3,25192322,65,1,3,2,10,107,2,182,1,...,1,0,0,0,0,1,0,1,0,1
4,80931528,45,3,1,0,0,119,2,18,0,...,0,0,1,1,0,0,1,0,1,0


In [5]:
validation_data.head()

,Age,Adults,Children,Babies,Reservation_Status,Discount_Rate,Room_Rate,Expected_stay_days,Reservation_gap,Gender_F,...,Deposit_type_No Deposit,Deposit_type_Non-Refundable,Deposit_type_Refundable,Booking_channel_Agent,Booking_channel_Direct,Booking_channel_Online,Required_Car_Parking_No,Required_Car_Parking_Yes,Use_Promotion_No,Use_Promotion_Yes
0,56,2,2,0,No-Show,15,192,2,195,0,...,1,0,0,1,0,0,1,0,0,1
1,60,2,2,0,Canceled,0,187,4,175,0,...,1,0,0,0,0,1,0,1,1,0
2,58,3,1,0,Canceled,10,227,1,193,1,...,1,0,0,0,1,0,1,0,0,1
3,23,1,2,0,Check-In,25,189,2,103,1,...,0,0,1,0,1,0,1,0,0,1
4,47,1,1,0,Check-In,10,218,1,92,1,...,1,0,0,0,0,1,0,1,0,1


### Seperating the columns of categorical and quantitative data

In [6]:
data_quan_cols = ['Age','Discount_Rate', 'Room_Rate','Expected_stay_days', 
                        'Reservation_gap', 'Adults', 'Children','Babies']

In [7]:
data_cat_cols = ['Gender', 'Ethnicity', 'Educational_Level', 'Income', 'Country_region',
                 'Hotel_Type', 'Meal_Type', 'Visted_Previously',
                 'Previous_Cancellations', 'Deposit_type', 'Booking_channel',
                 'Required_Car_Parking', 'Use_Promotion']

### Scaling the quantitative variables

In [8]:
train_data_quan = train_data[data_quan_cols]
validation_data_quan = validation_data[data_quan_cols]
test_data_quan = test_data[data_quan_cols]

In [9]:
sc = StandardScaler()

In [10]:
train_data_quan_scaled = sc.fit_transform(train_data_quan)
validation_data_quan_scaled = sc.transform(validation_data_quan)
test_data_quan_scaled = sc.transform(test_data_quan)

In [11]:
train_data_quan_scaled

array([[-0.25991554, -0.22266812,  0.97691768, ..., -0.28353117,
         0.35475481, -0.61317041],
       [ 0.32820944, -1.11506055,  0.22480302, ...,  0.56644415,
         1.73987514, -0.61317041],
       [-0.1292211 , -1.11506055, -1.27942629, ...,  0.56644415,
         1.73987514, -0.61317041],
       ...,
       [ 0.32820944, -0.66886434,  0.61225603, ..., -0.28353117,
        -1.03036552,  1.13107011],
       [ 1.37376495,  0.66972431, -0.41335487, ...,  0.56644415,
         0.35475481, -0.61317041],
       [ 0.26286222, -1.11506055,  0.79458685, ...,  0.56644415,
        -1.03036552, -0.61317041]])

In [12]:
type(train_data_quan_scaled)

numpy.ndarray

### Separating the categorical variables

In [13]:
cols_to_drop = data_quan_cols.copy()
cols_to_drop.append("Reservation_Status")
cols_to_drop

['Age',
 'Discount_Rate',
 'Room_Rate',
 'Expected_stay_days',
 'Reservation_gap',
 'Adults',
 'Children',
 'Babies',
 'Reservation_Status']

In [14]:
print(cols_to_drop)

['Age', 'Discount_Rate', 'Room_Rate', 'Expected_stay_days', 'Reservation_gap', 'Adults', 'Children', 'Babies', 'Reservation_Status']


In [15]:
train_data_cat = train_data.drop(cols_to_drop, axis=1)
validation_data_cat = validation_data.drop(cols_to_drop, axis=1)
test_data_cat = test_data.drop(data_quan_cols, axis=1)
train_data_cat.head()

,Gender_F,Gender_M,Ethnicity_African American,Ethnicity_Asian American,Ethnicity_Latino,Ethnicity_caucasian,Educational_Level_College,Educational_Level_Grad,Educational_Level_High-School,Educational_Level_Mid-School,...,Deposit_type_No Deposit,Deposit_type_Non-Refundable,Deposit_type_Refundable,Booking_channel_Agent,Booking_channel_Direct,Booking_channel_Online,Required_Car_Parking_No,Required_Car_Parking_Yes,Use_Promotion_No,Use_Promotion_Yes
0,1,0,0,0,1,0,0,1,0,0,...,1,0,0,0,0,1,0,1,0,1
1,1,0,0,0,1,0,0,0,0,1,...,0,0,1,0,0,1,0,1,1,0
2,1,0,0,0,0,1,0,1,0,0,...,1,0,0,0,0,1,0,1,1,0
3,0,1,1,0,0,0,1,0,0,0,...,0,0,1,1,0,0,0,1,0,1
4,1,0,0,0,1,0,0,0,1,0,...,1,0,0,0,1,0,1,0,0,1


In [16]:
train_data_cat_array = train_data_cat.to_numpy()
validation_data_cat_array = validation_data_cat.to_numpy()
test_data_cat_array = test_data_cat.to_numpy()
train_data_cat_array.shape

(27499, 38)

### Joining the pre-processed data

In [17]:
train_data_processed = np.concatenate((train_data_quan_scaled, train_data_cat_array), axis = 1)
validation_data_processed = np.concatenate((validation_data_quan_scaled, validation_data_cat_array), axis = 1)
test_data_processed = np.concatenate((test_data_quan_scaled, test_data_cat_array), axis = 1)

In [18]:
train_data_processed[0]

array([-0.25991554, -0.22266812,  0.97691768, -0.84878606, -0.88293639,
       -0.28353117,  0.35475481, -0.61317041,  1.        ,  0.        ,
        0.        ,  0.        ,  1.        ,  0.        ,  0.        ,
        1.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        1.        ,  0.        ,  0.        ,  1.        ,  0.        ,
        0.        ,  0.        ,  1.        ,  0.        ,  1.        ,
        0.        ,  0.        ,  1.        ,  0.        ,  1.        ,
        0.        ,  1.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  1.        ,  0.        ,  1.        ,  0.        ,
        1.        ])

In [19]:
test_data_quan_scaled.shape

(4318, 8)

In [20]:
test_data_cat_array.shape

(4318, 38)

In [21]:
validation_data_processed.shape

(2749, 46)

### Encoding the labels

In [22]:
train_y = train_data['Reservation_Status']
validation_y = validation_data['Reservation_Status']

In [23]:
le = preprocessing.LabelEncoder()

In [24]:
train_y = le.fit_transform(train_y)
validation_y = le.transform(validation_y)

In [25]:
validation_y.shape

(2749,)

In [26]:
unique = np.bincount(train_y)

In [27]:
print(unique)

[ 4134 21240  2125]


### Near Sampling...

In [28]:
#NearMiss
from imblearn.under_sampling import NearMiss

strategy = {1: round(21240*0.5), 0: 4134, 2: 2125}
undersample = NearMiss(sampling_strategy=strategy,version=3, n_neighbors=2)
X_near , y_near = undersample.fit_resample(train_data_processed,train_y)

/opt/anaconda3/envs/datastorm2/lib/python3.7/site-packages/imblearn/under_sampling/_prototype_selection/_nearmiss.py:176: UserWarning: The number of the samples to be selected is larger than the number of samples available. The balancing ratio cannot be ensure and all samples will be returned.
  "The number of the samples to be selected is larger"
/opt/anaconda3/envs/datastorm2/lib/python3.7/site-packages/imblearn/under_sampling/_prototype_selection/_nearmiss.py:176: UserWarning: The number of the samples to be selected is larger than the number of samples available. The balancing ratio cannot be ensure and all samples will be returned.
  "The number of the samples to be selected is larger"


In [29]:
unique_near = np.bincount(y_near)
print(unique_near)

[3005 5286 2125]


### Random Forest

In [30]:
# ran_for_params = {
#     'n_estimators' : range(1,200),
#     'criterion': ['gini', 'entropy'],
#     'max_depth': range(1,10),
#     'min_samples_leaf': range(1,10),
#     'min_samples_split': range(1,10),
#     'bootstrap' : [True, False]
# }

In [31]:
# random_forest_model = RandomForestClassifier()
# model_rand_fors = RandomizedSearchCV(random_forest_model, ran_for_params, cv=5,n_iter = 5000, n_jobs = -1, verbose = 1)
# model_rand_fors.fit(X_near, y_near)

In [32]:
# model_rand_fors.best_params_

In [33]:
# model_rand_fors.best_score_

In [34]:
# y_pred= model.predict(validation_data_processed)

In [35]:
# print(classification_report(validation_y, y_pred))

### Gradient Boosting

In [30]:
from sklearn.ensemble import GradientBoostingClassifier
model_grad_boost = GradientBoostingClassifier(random_state=0)

In [31]:
model_grad_boost.fit(X_near, y_near)

GradientBoostingClassifier(random_state=0)

In [32]:
y_pred= model_grad_boost.predict(validation_data_processed)

In [33]:
print(classification_report(validation_y, y_pred))

              precision    recall  f1-score   support

           0       0.25      0.10      0.14       741
           1       0.59      0.85      0.70      1610
           2       0.21      0.07      0.10       398

    accuracy                           0.54      2749
   macro avg       0.35      0.34      0.31      2749
weighted avg       0.44      0.54      0.46      2749



#### Hyper parameter tuning Gradient Boosting

In [40]:
grad_boost_param_grid = {
    'loss' : ['deviance', 'exponential'], 'max_features' : ['auto', 'sqrt', 'log2', None], 
    'learning_rate' : np.linspace(1.0, 0.0001, 25), 'n_estimators' : np.arange(1,125,5), 
    'max_depth' : np.arange(1,6)
    }

In [ ]:
model_grad_boost_base = GradientBoostingClassifier(random_state=0)
model_grad_boost_hp = GridSearchCV(model_grad_boost_base, grad_boost_param_grid, cv=5, n_jobs = -1, verbose = 1)
model_grad_boost_hp.fit(X_near, y_near)

In [ ]:
model_grad_boost_hp.best_params_

In [ ]:
model_grad_boost_hp.best_score_

In [46]:
model_grad_boost = GradientBoostingClassifier(n_estimators= 11,max_features= 'sqrt',max_depth= 2,
                                              loss= 'deviance',learning_rate= 0.3750625000000001,
                                              random_state=0)

In [47]:
model_grad_boost.fit(X_near, y_near)

GradientBoostingClassifier(learning_rate=0.3750625000000001, max_depth=2,
                           max_features='sqrt', n_estimators=11,
                           random_state=0)

In [48]:
y_pred= model_grad_boost.predict(validation_data_processed)

In [49]:
print(classification_report(validation_y, y_pred))

              precision    recall  f1-score   support

           0       0.30      0.04      0.07       741
           1       0.59      0.97      0.73      1610
           2       0.38      0.01      0.01       398

    accuracy                           0.58      2749
   macro avg       0.42      0.34      0.27      2749
weighted avg       0.48      0.58      0.45      2749



### XGBoost

In [173]:
xg_train = xgb.DMatrix(X_near, label=y_near)
xg_test = xgb.DMatrix(validation_data_processed, label=validation_y)

In [174]:
param = {}

In [175]:
# use softmax multi-class classification
param['objective'] = 'multi:softmax'
# scale weight of positive examples
param['eta'] = 0.15
param['max_depth'] = 14
param['nthread'] = -1
param['num_class'] = 3
param['max_delta_step'] = 1
#param['eval_metric'] = 'merror'
#param['scale_pos_weight'] = 0.5
#param['num_parallel_tree'] = 50

In [176]:
watchlist = [(xg_train, 'train'), (xg_test, 'test')]

In [177]:
num_round = 100

In [178]:
bst = xgb.train(param, xg_train, num_round, watchlist, 
                callbacks=[xgb.callback.EvaluationMonitor(show_stdv=False),xgb.callback.EarlyStopping(3)])

[01:15:06] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softmax' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[0]	train-mlogloss:1.01997	test-mlogloss:1.08259
[0]	train-mlogloss:1.01997	test-mlogloss:1.08259
[1]	train-mlogloss:0.94944	test-mlogloss:1.07009
[1]	train-mlogloss:0.94944	test-mlogloss:1.07009
[2]	train-mlogloss:0.88688	test-mlogloss:1.06080
[2]	train-mlogloss:0.88688	test-mlogloss:1.06080
[3]	train-mlogloss:0.83028	test-mlogloss:1.05369
[3]	train-mlogloss:0.83028	test-mlogloss:1.05369
[4]	train-mlogloss:0.78164	test-mlogloss:1.04876
[4]	train-mlogloss:0.78164	test-mlogloss:1.04876
[5]	train-mlogloss:0.73702	test-mlogloss:1.04566
[5]	train-mlogloss:0.73702	test-mlogloss:1.04566
[6]	train-mlogloss:0.69635	test-mlogloss:1.04199
[6]	train-mlogloss:0.69635	test-mlogloss:1.04199
[7]	train-mlogloss:0.65991	test-

In [179]:
y_pred = bst.predict(xg_test)

In [180]:
print(classification_report(validation_y, y_pred))

              precision    recall  f1-score   support

           0       0.28      0.23      0.25       741
           1       0.59      0.67      0.63      1610
           2       0.17      0.12      0.14       398

    accuracy                           0.47      2749
   macro avg       0.34      0.34      0.34      2749
weighted avg       0.44      0.47      0.45      2749



In [154]:
error_rate = np.sum(y_pred != validation_y) / validation_y.shape[0]
print('Test error using softmax = {}'.format(error_rate))

Test error using softmax = 0.42306293197526373


In [ ]:
bst_cv= xgb.cv(param, xg_train, num_round, nfold=5,
       metrics={'auc'}, seed=0,
       callbacks=[xgb.callback.EvaluationMonitor(show_stdv=False),
                  xgb.callback.EarlyStopping(3)])

In [ ]:
bst_cv.head()

### XGBoost with RFE

In [163]:
from sklearn.feature_selection import RFE

In [164]:
model = RandomForestClassifier() #insert the model you fitted
rfe = RFE(model, n_features_to_select = 15)
model = rfe.fit(X_near , y_near)

In [165]:
y_pred_rfe = model.predict(validation_data_processed)

In [166]:
print(classification_report(validation_y, y_pred_rfe))

              precision    recall  f1-score   support

           0       0.30      0.11      0.17       741
           1       0.58      0.88      0.70      1610
           2       0.08      0.01      0.02       398

    accuracy                           0.55      2749
   macro avg       0.32      0.33      0.30      2749
weighted avg       0.44      0.55      0.46      2749



In [155]:
xg_to_predict = xgb.DMatrix(test_data_processed)

In [167]:
y_pred_test = model.predict(test_data_processed)

In [168]:
print(y_pred_test)

[1 1 0 ... 1 1 1]


In [169]:
type(y_pred_test)

numpy.ndarray

In [170]:
dataset = pd.DataFrame({'Reservation_Status': y_pred_test})

In [171]:
dataset.head()

,Reservation_Status
0,1
1,1
2,0
3,0
4,1


In [172]:
dataset.to_csv('predictions_rfe.csv')